In [1]:
import matplotlib
matplotlib.use("Agg")
from neuralnilm.source import RealApplianceSource, MultiSource, SameLocation
import matplotlib.pyplot as plt
import numpy as np
import logging
import nilmtk
from datetime import timedelta

/home/dk3810/workspace/python/Lasagne/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [2]:
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [15]:
BUILDING_I = 1

appliance = 'dish washer'

N_SEQ_PER_BATCH = 8
FILENAME = '/data/mine/vadeec/merged/ukdale.h5'

WINDOWS_SELECTION = 'train'

if WINDOWS_SELECTION == 'short train':
    WINDOW_PER_BUILDING = {
        1: ("2013-04-12", "2013-04-18"),
        2: ("2013-05-22", "2013-06-01"),
        3: ("2013-02-27", "2013-03-10"),
        4: ("2013-03-09", "2013-04-01"),
        5: ("2014-06-29", "2014-07-10")
    }
elif WINDOWS_SELECTION == 'train':
    WINDOW_PER_BUILDING = {
        1: ("2013-04-12", "2014-12-15"),
        2: ("2013-05-22", "2013-10-03 06:16:00"),
        3: ("2013-02-27", "2013-04-01 06:15:05"),
        4: ("2013-03-09", "2013-09-24 06:15:14"),
        5: ("2014-06-29", "2014-09-01")
    }
elif WINDOWS_SELECTION == 'test':
    WINDOW_PER_BUILDING = {
        1: ("2014-12-15", "2014-12-22"),
        2: ("2013-10-03 06:16:00", None),
        3: ("2013-04-01 06:15:05", None),
        4: ("2013-09-24 06:15:14", None),
        5: ("2014-09-01", "2014-09-07")
    }

INPUT_STATS = {
    'mean': np.array([ 297.87216187], dtype=np.float32),
    'std': np.array([ 374.43884277], dtype=np.float32)
}

if appliance == 'microwave':
    SEQ_LENGTH = 288
    APPLIANCES = [
        'microwave',
        ['fridge freezer', 'fridge', 'freezer'],
        'dish washer',
        'kettle',
        ['washer dryer', 'washing machine']
    ]
    MAX_APPLIANCE_POWERS = [3000,  300, 2500, 3100, 2500]
    ON_POWER_THRESHOLDS  = [ 200,   50,   10, 2000,   20]
    MIN_ON_DURATIONS     = [  12,   60, 1800,   12, 1800]
    MIN_OFF_DURATIONS    = [  30,   12, 1800,    0,  160]

elif appliance == 'washing machine':
    SEQ_LENGTH = 1536
    APPLIANCES = [
        ['washer dryer', 'washing machine'],
        ['fridge freezer', 'fridge', 'freezer'],
        'dish washer',
        'kettle',
        'microwave'            
    ]
    MAX_APPLIANCE_POWERS = [2500,  300, 2500, 3100, 3000]
    ON_POWER_THRESHOLDS  = [  20,   50,   10, 2000,  200]
    MIN_ON_DURATIONS     = [1800,   60, 1800,   12,   12]
    MIN_OFF_DURATIONS    = [ 160,   12, 1800,    0,   30]

elif appliance == 'fridge':
    SEQ_LENGTH = 1024
    APPLIANCES = [
        ['fridge freezer', 'fridge', 'freezer'],
        ['washer dryer', 'washing machine'],
        'dish washer',
        'kettle',
        'microwave'            
    ]
    MAX_APPLIANCE_POWERS = [ 300, 2500, 2500, 3100, 3000]
    ON_POWER_THRESHOLDS  = [  50,   20,   10, 2000,  200]
    MIN_ON_DURATIONS     = [  60, 1800, 1800,   12,   12]
    MIN_OFF_DURATIONS    = [  12,  160, 1800,    0,   30]

elif appliance == 'kettle':
    SEQ_LENGTH = 128
    APPLIANCES = [
        'kettle',
        ['fridge freezer', 'fridge', 'freezer'],
        ['washer dryer', 'washing machine'],
        'dish washer',
        'microwave'
    ]
    MAX_APPLIANCE_POWERS = [3100,  300, 2500, 2500, 3000]
    ON_POWER_THRESHOLDS  = [2000,   50,   20,   10,  200]
    MIN_ON_DURATIONS     = [  12,   60, 1800, 1800,   12]
    MIN_OFF_DURATIONS    = [   0,   12,  160, 1800,   30]

elif appliance == 'dish washer':
    SEQ_LENGTH = 1536
    APPLIANCES = [
        'dish washer',
        ['fridge freezer', 'fridge', 'freezer'],
        ['washer dryer', 'washing machine'],
        'kettle',
        'microwave'
    ]
    MAX_APPLIANCE_POWERS = [2500,  300, 2500, 3100, 3000]
    ON_POWER_THRESHOLDS  = [  10,   50,   20, 2000,  200]
    MIN_ON_DURATIONS     = [1800,   60, 1800,   12,   12]
    MIN_OFF_DURATIONS    = [1800,   12,  160,    0,   30]

TARGET_APPLIANCE = APPLIANCES[0]
MAX_TARGET_POWER = MAX_APPLIANCE_POWERS[0]
ON_POWER_THRESHOLD = ON_POWER_THRESHOLDS[0]
MIN_ON_DURATION = MIN_ON_DURATIONS[0]
MIN_OFF_DURATION = MIN_OFF_DURATIONS[0]

In [16]:
source = SameLocation(
    logger=logger,
    filename=FILENAME,
    target_appliance=TARGET_APPLIANCE,
    window_per_building=WINDOW_PER_BUILDING,
    seq_length=SEQ_LENGTH,
    train_buildings=[BUILDING_I],
    validation_buildings=[BUILDING_I],
    n_seq_per_batch=N_SEQ_PER_BATCH,
    load_mains=False,
    on_power_threshold=ON_POWER_THRESHOLD,
    min_on_duration=MIN_ON_DURATION,
    min_off_duration=MIN_OFF_DURATION
)

INFO:test:Setting window for building 1 to (start=2013-04-12, end=2014-12-15)
INFO:test:Loaded 197 dish washer activations from house 1.


In [11]:
seq_lengths = [(activation.index[-1] - activation.index[0])
               for activation in source.activations[BUILDING_I]]
max_duration = max(seq_lengths)
print("Max duration  :", max_duration)
print("Min input size:", max_duration.total_seconds() / 6)
print("Mean duration :", np.mean(seq_lengths))
print("Min duration  :", np.min(seq_lengths))
index_with_max_duration = np.argmax(seq_lengths)
print("Index with max duration :", index_with_max_duration)

Max duration  : 0 days 00:11:36
Min input size: 116.0
Mean duration : 0 days 00:02:03.484485
Min duration  : 0 days 00:00:24
Index with max duration : 2720


In [12]:
print("Max power  :", max([activation.max() for activation in source.activations[BUILDING_I]]))
print("Mean power :", np.mean([activation.mean() for activation in source.activations[BUILDING_I]]))

Max power  : 3948.0
Mean power : 1955.72096691


In [13]:
dataset = nilmtk.DataSet(FILENAME)
dataset.set_window()

elec = dataset.buildings[BUILDING_I].elec
meter = elec[TARGET_APPLIANCE]
meter

ElecMeter(instance=10, building=1, dataset='UK-DALE', appliances=[Appliance(type='kettle', instance=1), Appliance(type='food processor', instance=1), Appliance(type='toasted sandwich maker', instance=1)])

In [14]:
elec.mains().get_timeframe()

TimeFrame(start='2013-03-17 19:12:43.100000+00:00', end='2015-01-05 06:27:12+00:00', empty=False)

In [434]:
border = timedelta(minutes=10)

def plot_activation(activation_i):
    activation = source.activations[BUILDING_I][activation_i]
    timeframe = nilmtk.TimeFrame(activation.index[0]-border, activation.index[-1]+border)
    powers = {}
    elec = dataset.buildings[BUILDING_I].elec
    for meter in elec.submeters().meters:
        power = meter.power_series_all_data(sections=[timeframe])
        if power is not None and power.max() > 10:
            powers[meter.dominant_appliance().label(pretty=True)] = power
    
    fig, axes = plt.subplots(nrows=3, sharex=True)
    axes[0].set_title('Activation')
    axes[0].plot(activation.index, activation.values)

    axes[1].set_title('All active appliances')
    for meter_instance, power in powers.iteritems():
        axes[1].plot(power.index, power.values, label=meter_instance)
    axes[1].legend(fancybox=True, framealpha=0.5, prop={'size': 6})

    axes[2].set_title('Mains {}'.format(timeframe.start.date()))
    mains = elec.mains().power_series_all_data(sections=[timeframe], sample_period=6)
    if mains is not None:
        axes[2].plot(mains.index, mains.values)
    
    fig.tight_layout()
    fname = 'building{}_activation{}.png'.format(BUILDING_I, activation_i)
    plt.savefig(fname, dpi=300)
    plt.close(fig)

In [435]:
from sys import stdout
plot_activation(index_with_max_duration)
n = len(source.activations[BUILDING_I])
for i in range(n):
    print("\r", i, "/", n, end="")
    stdout.flush()
    plot_activation(i)

 52 / 90

Exception KeyboardInterrupt in <function remove at 0x7effac36e8c0> ignored


TypeError: sequence item 0: expected string, tuple found